### 1.0 Libraries and Directories

In [11]:
import ee
import geemap
import pprint as pp

In [12]:
ee.Authenticate()
ee.Initialize()

In [15]:
glad = ee.ImageCollection('projects/glad/water/individualMonths')

rois = ee.data.listAssets('projects/alpod-412314/assets/ROIs/')
rois_ids = [roi['id'] for roi in rois['assets']]
rois_ids.remove('projects/alpod-412314/assets/ROIs/all_regions')
pp.pp(rois_ids)

['projects/alpod-412314/assets/ROIs/AKCP_roi',
 'projects/alpod-412314/assets/ROIs/MRD_TUK_Anderson_roi',
 'projects/alpod-412314/assets/ROIs/MRD_roi',
 'projects/alpod-412314/assets/ROIs/TUK_roi',
 'projects/alpod-412314/assets/ROIs/YKdelta_roi',
 'projects/alpod-412314/assets/ROIs/YKflats_roi',
 'projects/alpod-412314/assets/ROIs/anderson_plain_roi']


### 2.0 Combine all the GLAD observations for June & August for 20 years. 

In [14]:
aug_pattern = r'\d{4}_08'
june_pattern = r'\d{4}_06'

aug_img
test_img = glad.filter(ee.Filter.eq('system:index', test_id)).first()

pp.pp(test_img.getInfo())

{'type': 'Image',
 'bands': [{'id': 'wp',
            'data_type': {'type': 'PixelType',
                          'precision': 'int',
                          'min': 0,
                          'max': 255},
            'dimensions': [1432001, 560001],
            'crs': 'EPSG:4326',
            'crs_transform': [0.00025, 0, -179, 0, -0.00025, 80]}],
 'version': 1612794707629899,
 'id': 'projects/glad/water/individualMonths/2001_08',
 'properties': {'system:time_start': 997833600000,
                'system:footprint': {'type': 'LinearRing',
                                     'coordinates': [[-180, -90],
                                                     [180, -90],
                                                     [180, 90],
                                                     [-180, 90],
                                                     [-180, -90]]},
                'system:asset_size': 8723833704,
                'system:index': '2001_08'}}


In [16]:
def export_rois(roi_id, target_img):
    
    roi_name_temp = roi_id.split('/')[-1]
    roi_name = roi_name_temp.split('_roi')[0]
    feature = ee.FeatureCollection(roi_id).first()
    bounds = feature.geometry()

    clipped_img = target_img.clip(bounds)

    export_params = {
        'image': clipped_img,
        'description': f'glad_{roi_name}_test', 
        'fileNamePrefix': f'glad_{roi_name}_test',
        'folder': 'glad_exports', 
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': bounds,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e13
    }

    ee.batch.Export.image.toDrive(**export_params).start()

In [17]:
for roi_id in rois_ids:
    export_rois(roi_id, test_img)

In [27]:
print(roi_id)

feature = ee.FeatureCollection(roi_id).first()
bounds = feature.geometry()

min_max = test_img.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=bounds,
    scale=30,
    maxPixels=1e13
).getInfo()

projects/alpod-412314/assets/ROIs/anderson_plain_roi


In [28]:
print(min_max)

{'wp_max': 100, 'wp_min': 0}


In [29]:
pp.pp(glad.getInfo())

{'type': 'ImageCollection',
 'bands': [],
 'version': 1647386366866962,
 'id': 'projects/glad/water/individualMonths',
 'features': [{'type': 'Image',
               'bands': [{'id': 'wp',
                          'data_type': {'type': 'PixelType',
                                        'precision': 'int',
                                        'min': 0,
                                        'max': 255},
                          'dimensions': [1432001, 560001],
                          'crs': 'EPSG:4326',
                          'crs_transform': [0.00025,
                                            0,
                                            -179,
                                            0,
                                            -0.00025,
                                            80]}],
               'version': 1612794604895389,
               'id': 'projects/glad/water/individualMonths/1999_01',
               'properties': {'system:time_start': 916358400000,
  